In [2]:
import numpy as np
import pandas as pd
import glob
import os
from tqdm.notebook import tqdm
tqdm.pandas()

# import swifter

In [3]:
def prepare_tweets_df(df, stock_sym):
    tweets_dict = {
        'stock': [],
        'date': [],
        'tweets': []
    }
    for _, row in tqdm(df.iterrows()):
        date = row['date']
        tweets = row.iloc[2:61].values
        # print(len(tweets))
        tweets_dict['tweets'].extend(tweets)
        tweets_dict['stock'].extend([stock_sym for i in range(0, len(tweets))])
        tweets_dict['date'].extend([date for i in range(0, len(tweets))])
    return pd.DataFrame(tweets_dict)

In [4]:
tweet_files_path = os.getcwd() + '/dataset/Tweets/'
stocks_files_path = os.getcwd() + '/dataset/Stocks/'

stock_names = ['AAPL', 'AMT', 'AMZN', 'APRN', 'C', 'DIS', 'FB', 'GE', 'GOOGL', 'HTZ', 'LUV', 'MSFT', 'NFLX', 'SBUX', 'TSLA']

tweets_df_list = []
for stock_name in stock_names:
    tweets_df_list.append(prepare_tweets_df(pd.read_csv(tweet_files_path + stock_name + '.csv'), stock_name))

tweets_df = pd.concat(tweets_df_list, axis = 0)

# tweets_df = pd.concat([prepare_tweets_df(pd.read_csv(tweet_files_path + stock_name + '.csv'), stock_name) for stock_name in stock_names], axis = 0)

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

...

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [5]:
tweets_df.sample(20)
print(tweets_df.shape)
print(tweets_df.isna().sum())
tweets_df = tweets_df.dropna()
# print(tweets_df.duplicated().sum())
print(len(tweets_df)-len(tweets_df.drop_duplicates()))
tweets_df = tweets_df.drop_duplicates()
print(tweets_df.isna().sum())
print(tweets_df.shape)

(1269444, 3)
stock           0
date            0
tweets    1002313
dtype: int64
17936
stock     0
date      0
tweets    0
dtype: int64
(249195, 3)


In [6]:
# Due to incorrect formatting of the source csv files, some strings are available in date column. 
# So we are forcing those values to be NaT
# Also dropping those rows
tweets_df['date'] = pd.to_datetime(tweets_df['date'], errors='coerce')
print(tweets_df.isna().sum())
tweets_df = tweets_df.dropna()
print(tweets_df.shape)

stock       0
date      170
tweets      0
dtype: int64
(249025, 3)


In [7]:
# !pip install -q -U transformers
# !pip install -U torch torchvision torchaudio

In [8]:
# https://huggingface.co/cardiffnlp/twitter-roberta-base-sentiment-latest?text=Covid+cases+are+increasing+fast%21
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
import numpy as np
from scipy.special import softmax

# Preprocess text (username and link placeholders)
def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)
# PT
model = AutoModelForSequenceClassification.from_pretrained(MODEL)


Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [9]:
def get_sentiment(row):
    text = preprocess(row['tweets'])
    encoded_input = tokenizer(text, return_tensors='pt')
    output = model(**encoded_input)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    row['negative'] = scores[0]
    row['neutral'] = scores[1]
    row['positive'] = scores[2]
    return row

In [15]:
# %%time 

# tweets_df_1 = tweets_df.head(1000).reset_index()
# tweets_df_2 = tweets_df_1.progress_apply(get_sentiment, axis = 1)

  0%|          | 0/1000 [00:00<?, ?it/s]

CPU times: user 19min 39s, sys: 2min 36s, total: 22min 15s
Wall time: 21min 1s


In [19]:
# %%time
# import multiprocessing as mp
# import dask.dataframe as dd #Dask Multiprocessing 
# import os
# os.environ["TOKENIZERS_PARALLELISM"] = "true"
# n_workers = mp.cpu_count()
# print(n_workers)
# tweets_df_1 = tweets_df.head(10).reset_index()
# ddata = dd.from_pandas(tweets_df_1, npartitions=n_workers)
# new_df = ddata.map_partitions(lambda df: df.apply(get_sentiment, axis=1)).compute(scheduler='processes')  



10
CPU times: user 9.65 s, sys: 6.73 s, total: 16.4 s
Wall time: 27.6 s


In [22]:
display(new_df.head())
print(new_df.shape)

,index,stock,date,tweets,negative,neutral,positive
0,0,AAPL,2019-07-15,Top 5 Apple Prime Day deals http://dlvr.it/R8...,0.004799,0.442696,0.552505
1,1,AAPL,2019-07-15,CCI Buy Signals : $AAPL $GDXJ $GH $ZS $JNUG,0.016513,0.858549,0.124938
2,2,AAPL,2019-07-15,Traders checkout these trade plans given over ...,0.004786,0.595695,0.399519
3,3,AAPL,2019-07-15,272 stocks \nMoved Above Upper Price Channel...,0.010387,0.797119,0.192493
4,4,AAPL,2019-07-15,Watch what happened at 3:23 in @NaviAvatar1's ...,0.005847,0.707266,0.286886


(10, 7)


In [42]:
%%time

from tqdm.contrib.concurrent import process_map
import multiprocessing as mp
from functools import partial
import os
os.environ["TOKENIZERS_PARALLELISM"] = "true"


tweets_df_1 = tweets_df.head(100).reset_index()
n_workers = 2 * mp.cpu_count()
p = mp.Pool(n_workers)
batch = round(len(tweets_df_1)/n_workers)

# get_sentiment_map = partial(get_sentiment, axis = 1)
tweets_df_2 = p.map(get_sentiment, tweets_df_1.iterrows())

Process SpawnPoolWorker-269:
Process SpawnPoolWorker-270:
Traceback (most recent call last):
  File "/Users/ibrahim/opt/anaconda3/envs/ml/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/Users/ibrahim/opt/anaconda3/envs/ml/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/ibrahim/opt/anaconda3/envs/ml/lib/python3.10/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/Users/ibrahim/opt/anaconda3/envs/ml/lib/python3.10/multiprocessing/queues.py", line 367, in get
    return _ForkingPickler.loads(res)
AttributeError: Can't get attribute 'get_sentiment' on <module '__main__' (built-in)>
Process SpawnPoolWorker-272:
Traceback (most recent call last):
  File "/Users/ibrahim/opt/anaconda3/envs/ml/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/Users/ibrahim/opt/anaconda3/envs/ml/lib/python3.10/multiprocessing/process

KeyboardInterrupt: 

py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/Users/ibrahim/opt/anaconda3/envs/ml/lib/python3.10/multiprocessing/queues.py", line 364, in get
    with self._rlock:
  File "/Users/ibrahim/opt/anaconda3/envs/ml/lib/python3.10/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt
KeyboardInterrupt
Traceback (most recent call last):
  File "/Users/ibrahim/opt/anaconda3/envs/ml/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/Users/ibrahim/opt/anaconda3/envs/ml/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/ibrahim/opt/anaconda3/envs/ml/lib/python3.10/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/Users/ibrahim/opt/anaconda3/envs/ml/lib/python3.10/multiprocessing/queues.py", line 364, in get
    with self._rlock:
  File "/Users/ibrahim/opt/anaconda3/envs/ml/lib/python3

In [17]:
tweets_df_1.head(10)

,stock,date,tweets,negative,neutral,positive
0,AAPL,2019-07-15,Top 5 Apple Prime Day deals http://dlvr.it/R8...,0.004799,0.442696,0.552505
1,AAPL,2019-07-15,CCI Buy Signals : $AAPL $GDXJ $GH $ZS $JNUG,0.016513,0.858549,0.124938
2,AAPL,2019-07-15,Traders checkout these trade plans given over ...,0.004786,0.595695,0.399519
3,AAPL,2019-07-15,272 stocks \nMoved Above Upper Price Channel...,0.010387,0.797119,0.192493
4,AAPL,2019-07-15,Watch what happened at 3:23 in @NaviAvatar1's ...,0.005847,0.707266,0.286886
5,AAPL,2019-07-15,New US House Bill Would Ban Big Tech From 'Ban...,0.082856,0.858939,0.058206
6,AAPL,2019-07-15,Watch what happened at 0:49 in @NaviAvatar1's ...,0.006015,0.738211,0.255774
7,AAPL,2019-07-15,Monday: Up 6.79%. Cut losses on $PZZA 7/19 Cal...,0.013104,0.381210,0.605686
8,AAPL,2019-07-15,CarPlay now works with audiobooks borrowed fro...,0.004061,0.456838,0.539101
9,AAPL,2019-07-15,RT stock_family: CIAZEKACTrader Shhhh don't te...,0.527637,0.448191,0.024171


## Stock price processing

In [24]:
def prepare_stocks_df(df, stock_name):
    df['stock'] = [stock_name for i in range(len(df))]
    return df

In [29]:
stocks_df_list = []
for stock_name in stock_names:
    stocks_df_list.append(prepare_stocks_df(pd.read_csv(stocks_files_path + stock_name + '.csv'), stock_name))

stocks_df = pd.concat(stocks_df_list, axis = 0)
stocks_df.columns = ['_'.join(column.lower().split()) for column in stocks_df.columns]

In [30]:
print(stocks_df.shape)
print(stocks_df.isna().sum())
display(stocks_df.head(10))

(3763, 8)
date         0
open         0
high         0
low          0
close        0
adj_close    0
volume       0
stock        0
dtype: int64


,date,open,high,low,close,adj_close,volume,stock
0,2019-07-15,204.089996,205.869995,204.000000,205.210007,202.789810,16947400,AAPL
1,2019-07-16,204.589996,206.110001,203.500000,204.500000,202.088181,16866800,AAPL
2,2019-07-17,204.050003,205.089996,203.270004,203.350006,200.951736,14107500,AAPL
3,2019-07-18,204.000000,205.880005,203.699997,205.660004,203.234497,18582200,AAPL
4,2019-07-19,205.789993,206.500000,202.360001,202.589996,200.200699,20929300,AAPL
5,2019-07-22,203.649994,207.229996,203.610001,207.220001,204.776108,22277900,AAPL
6,2019-07-23,208.460007,208.910004,207.289993,208.839996,206.376984,18355200,AAPL
7,2019-07-24,207.669998,209.149994,207.169998,208.669998,206.209015,14991600,AAPL
8,2019-07-25,208.889999,209.240005,206.729996,207.020004,204.578476,13909600,AAPL
9,2019-07-26,207.479996,209.729996,207.139999,207.740005,205.289978,17618900,AAPL


In [32]:
stocks_df['date'] = pd.to_datetime(stocks_df['date'])

In [33]:
data_df = pd.merge(tweets_df, stocks_df, how = 'inner', on = ['date', 'stock'])

In [37]:
print(data_df.shape)
display(tweets_df.tail(20))
display(stocks_df.tail(20))
display(data_df.sample(20))

(180360, 9)


,stock,date,tweets
21219,TSLA,2020-07-08,The wholesale shipment number of Made-in-China...
21220,TSLA,2020-07-08,You’re insulting me for having no money and im...
21221,TSLA,2020-07-08,"My brother's fevered imagination, which is the..."
21222,TSLA,2020-07-08,RT elonmusk: vincent13031925 teslacn EvaFoxU T...
21223,TSLA,2020-07-08,I want a tesla
21224,TSLA,2020-07-08,Ok well that's interesting\n\nSince last Septe...
21225,TSLA,2020-07-08,BGI FLYING in afterhours 🚀🚀🚀🚀\n\nMONSTER EARNI...
21226,TSLA,2020-07-08,Tesla hires guy behind Apple Pay to lead Super...
21227,TSLA,2020-07-08,Elon Musk would go without sleep for days and ...
21228,TSLA,2020-07-08,@elonmusk backs dogecoin


,date,open,high,low,close,adj_close,volume,stock
233,2020-06-16,1011.849976,1012.880005,962.390015,982.130005,982.130005,14051100,TSLA
234,2020-06-17,987.710022,1005.000000,982.570007,991.789978,991.789978,9869400,TSLA
235,2020-06-18,1003.000000,1019.200012,994.469971,1003.960022,1003.960022,9751900,TSLA
236,2020-06-19,1012.780029,1015.969971,991.340027,1000.900024,1000.900024,8679700,TSLA
237,2020-06-22,999.950012,1008.880005,990.020020,994.320007,994.320007,6362400,TSLA
238,2020-06-23,998.880005,1012.000000,994.010010,1001.780029,1001.780029,6365300,TSLA
239,2020-06-24,994.109985,1000.880005,953.140015,960.849976,960.849976,10959600,TSLA
240,2020-06-25,954.270020,985.979980,937.150024,985.979980,985.979980,9254500,TSLA
241,2020-06-26,994.780029,995.000000,954.869995,959.739990,959.739990,8854900,TSLA
242,2020-06-29,969.010010,1010.000000,948.520020,1009.349976,1009.349976,9026400,TSLA


,stock,date,tweets,open,high,low,close,adj_close,volume
124054,MSFT,2019-08-01,BUY $pins most misunderstood stock in Tech\n\n...,137.000000,140.940002,136.929993,138.059998,136.381760,40557500
124102,MSFT,2019-08-02,#estate asset watch - top money flow Friday - ...,138.089996,138.320007,135.259995,136.899994,135.235855,30791600
154730,SBUX,2019-09-16,#BillAckman's #PershingSquareHoldings (#OTCPK:...,90.019997,90.449997,89.379997,89.419998,88.064789,6339200
77580,FB,2020-03-11,"Dumbest statement I ever heard: ""Social stocks...",174.009995,175.880005,167.660004,170.240005,170.240005,20373900
99832,GOOGL,2019-10-04,$GOOGL getting really close to my 1209 number....,1194.290039,1212.459961,1190.969971,1210.959961,1210.959961,1214100
1285,AAPL,2019-08-14,Words of gold. Remembering John (Jack) C. Bogl...,203.160004,206.440002,202.589996,202.750000,201.120071,36547400
138159,NFLX,2019-07-23,$NFLX getting to oversold bounce level. Watch...,311.440002,313.500000,306.000000,307.299988,307.299988,9171100
65022,DIS,2020-04-22,#COVID19 #COVID #COVIDー19 $brka $aapl $wmt $di...,101.800003,102.629997,100.889999,100.989998,100.989998,11492000
11313,AAPL,2020-04-22,Why Apple investors should be worried by AT&T’...,273.609985,277.899994,272.200012,276.100006,275.354614,29264300
104827,GOOGL,2020-02-12,#FAANG Stocks Overview: https://techniquant.c...,1515.859985,1520.300049,1508.670044,1518.630005,1518.630005,1552100


In [39]:
import datetime


today = datetime.date.today()
today - datetime.timedelta(days=7)

datetime.date(2022, 10, 23)